First download the model weights and parameters, and install caffe

In [1]:
import caffe

In [2]:
caffe.set_mode_cpu()

In [3]:
#load the model
net = caffe.Net('model/VGG_CNN_S_deploy.prototxt',
                'model/VGG_CNN_S.caffemodel',
                caffe.TEST)

In [4]:
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
transformer.set_transpose('data', (2,0,1))
transformer.set_channel_swap('data', (2,1,0))
transformer.set_raw_scale('data', 255.0)
net.blobs['data'].reshape(1,3,224,224)
im = caffe.io.load_image('model/cat.jpg')
net.blobs['data'].data[...] = transformer.preprocess('data', im)

In [5]:
out = net.forward()

In [6]:
top_k = net.blobs['prob'].data[0].flatten().argsort()[-1:-6:-1]
print(top_k)

[331 285 287 281 330 278 277 282 335 332 937 333 998 284]


In [ ]:
def generateHDF5FromText2(label_num):
    print('\nplease wait...')

    HDF5_FILE = ['hdf5_train.h5',
                 'hdf5_test.h5']

    LIST_FILE = ['train.txt','test.txt']
    for kk, list_file in enumerate(LIST_FILE):
        path_list = []
        label_list = []
        with open(script_dir+"/"+list_file, buffering=1) as hosts_file:
            for line in hosts_file:
                line = line.rstrip()
                array = line.split(' ')
                lab = int(array[1])
                label_list.append(lab)
                path_list.append(array[0])
            print(len(path_list), len(label_list))
            datas = np.zeros((len(path_list),3,640,640),dtype='f4') 
            labels = np.zeros((len(path_list), 5),dtype="f4")

            datas = np.zeros((len(path_list),3,227,227)) 
            labels = np.zeros((len(path_list), 1))

            for ii, _file in enumerate(path_list):
                # feed image to HDF5
                im = Image.open(_file)
                im = im.resize((227, 227))
                img = np.array(im.getdata()).reshape(227,227, 3)
                img = np.transpose( img , (2,0,1))
                datas[ii, :, :, :] = np.array(img).astype(np.float32) / 255
            # feed label to HDF5
            label_array = label_list[ii]
            labels[0, :] = np.array( label_array ).astype(np.int)

            with h5py.File(script_dir+"/data2/"+HDF5_FILE[kk], 'w') as f:
                f['data'] = datas.astype(np.float32)
                f['label'] = labels.astype(np.float32)
                f.close()